In [304]:
using Pkg
Pkg.activate(".")

  Activating project at `c:\Users\fefif\Desktop\PhD\Progetti\MatrixProductBP.jl`


In [305]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs, Statistics, Random, LinearAlgebra, TensorTrains, SparseArrays
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using TensorTrains: summary_compact
using SparseArrays
using JLD2
include("bp_fourier.jl");

In [306]:
seed = 1
rng = MersenneTwister(seed)

T = 4
# N = 3
# A = [0 1 1; 
#      1 0 0;
#      1 0 0;]
# N = 4
# A = [0 1 1 1;
#      1 0 0 1;
#      1 0 0 1;
#      1 1 1 0]
# !!! Here the calculations are wrong for edge 1->2 !!!
# N = 5
# A = [0 1 1 1 1; 
#      1 0 0 0 0; 
#      1 0 0 0 0; 
#      1 0 0 0 0; 
#      1 0 0 0 0]
# N = 6
# A = [0 1 1 1 0 0; 
#      1 0 0 0 1 0; 
#      1 0 0 0 0 1; 
#      1 0 0 0 0 0; 
#      0 1 0 0 0 0; 
#      0 0 1 0 0 0]
# N = 7
# A = [0 1 1 1 1 0 1;
#      1 0 0 0 1 0 0; 
#      1 0 0 0 0 1 0; 
#      1 0 0 0 0 0 0; 
#      1 1 0 0 0 0 0; 
#      0 0 1 0 0 0 0; 
#      1 0 0 0 0 0 0]
# N = 8
# A = [0 1 1 1 0 0 1 0;
#      1 0 0 0 1 0 0 1; 
#      1 0 0 0 0 1 0 0; 
#      1 0 0 0 0 0 0 0; 
#      0 1 0 0 0 0 0 0; 
#      0 0 1 0 0 0 0 0; 
#      1 0 0 0 0 0 0 0;
#      0 1 0 0 0 0 0 0]

# g = IndexedBiDiGraph(A)

N = 10
c = 3
gg = erdos_renyi(N, c/N; seed)
g = IndexedBiDiGraph(gg)
A = zeros(Int,N,N)
for i in CartesianIndices(A)
     if !iszero(g.A[i])
         A[i] = 1
     end
end
β = 1.0
J = 1
h = 0.0
m⁰ = 1.0
K = 100

svd_trunc=TruncBond(4)

A

10×10 Matrix{Int64}:
 0  0  0  0  0  0  1  1  0  1
 0  0  1  0  0  0  1  0  0  0
 0  1  0  1  0  1  1  0  1  0
 0  0  1  0  1  1  0  0  0  0
 0  0  0  1  0  0  1  1  0  0
 0  0  1  1  0  0  0  1  0  1
 1  1  1  0  1  0  0  1  1  1
 1  0  0  0  1  1  1  0  0  0
 0  0  1  0  0  0  1  0  0  0
 1  0  0  0  0  1  1  0  0  0

In [307]:
w = [fill(IntegerGlauberFactor([J*A[j,i] for j in inneighbors(g,i)], h, β), T+1) for i in vertices(g)]
ϕᵢ = [t == 0 ? [(1-m⁰)/2, (1+m⁰)/2] : ones(2) for t in 0:T]
bp = mpbp(g, w, fill(2,N), T; ϕ=fill(ϕᵢ,N))

for μ in bp.μ
    for a in μ
        a.=rand.(rng)
        # a.=1.0
    end
end
bp_fourier = deepcopy(bp)
iterate!(bp, maxiter=1; svd_trunc)

# msg = deepcopy(bp.μ)
# jldsave("messages.jld2"; msg)

# bp_fourier = deepcopy(bp)
iterate_fourier!(bp_fourier,K, maxiter=1, σ=1/100; svd_trunc)
;

In [308]:
for t in 1:T+1
    # display(t)
    # display(bp.μ[7][t])
    # display(bp_fourier.μ[7][t])
end

In [309]:
iterate!(bp,maxiter=5; svd_trunc)

Running MPBP: iter 2    Time: 0:00:00

Running MPBP: iter 3    Time: 0:00:00

Running MPBP: iter 4    Time: 0:00:00

Running MPBP: iter 5    Time: 0:00:01

(5, CB_BP{ProgressMeter.ProgressUnknown, MatrixProductBP.var"#153#156"}(ProgressMeter.ProgressUnknown(false, false, 0, ProgressMeter.ProgressCore(:green, "Running MPBP: iter", 0.1, true, 0, VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(Base.Libc.WindowsRawSocket(0x0000000000000314) open, 0 bytes waiting)), VSCodeServer.io_send_callback), false, 1, 5, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (2278191112456, 2278191025064, 140732428818288)), 2, 5, true, false, 1.734690598525e9, 1.734690599563e9, 1.734690598525e9)), [[[2.0000000000000004, 1.3962748922517556, 1.2512455982135078, 1.4398183349948888, 1.254966312745844], [2.0, 1.1624517352436172, 1.0571995336666546, 1.9862614107620502, 1.289571470828798], [2.0, 1.8776458571167745, 1.9307376060121948, 1.3307461346186709, 1.5886285208426656], [2.0, 1

In [310]:
iterate_fourier!(bp_fourier,K, maxiter=5, σ=1/100; svd_trunc)
;

In [311]:
import MatrixProductBP: marginalize
function marginalize(A::MPEM2{F}, var) where F
    if var==1
        return MPEM1{F}([@tullio b[m, n, xi] := a[m, n, xi, xj] for a in A]; z = A.z)
    elseif var==2
        return MPEM1{F}([@tullio b[m, n, xj] := a[m, n, xi, xj] for a in A]; z = A.z)
    end
end;

In [312]:
er = []
ptr = []
for e in edges(bp.g)
    μ_marg = marginalize(bp.μ[idx(e)],1)
    margs = marginals(μ_marg)
    μ_fourier_marg = marginalize(bp_fourier.μ[idx(e)],1)
    margs_fourier = marginals(μ_fourier_marg)
    for t in 1:T+1
        marg = margs[t]
        marg_fourier = margs_fourier[t]
        ind = argmax(abs.(marg))
        push!(er, abs(marg_fourier[ind]-marg[ind])/abs(marg[ind]))
        push!(ptr, (e, t))
    end
end

display(maximum(er))
display(ptr[argmax(er)])

0.042653831618361955

(Indexed Edge 5 => 8 with index 16, 3)

In [313]:
node = 3
time = 3

ed = inedges(bp.g, node) |> first
@show ed
μ = bp.μ[idx(ed)]
μ_marg = marginalize(μ,1)
marg = marginals(μ_marg)[time]

ed_fourier = inedges(bp_fourier.g, node) |> first
@show ed_fourier
μ_fourier = bp_fourier.μ[idx(ed_fourier)]
μ_fourier_marg = marginalize(μ_fourier,1)
marg_fourier = marginals(μ_fourier_marg)[time]

display(marg)
display(marg_fourier);

2-element Vector{Float64}:
 0.2589935066133612
 0.7410064933866389

2-element Vector{Float64}:
 0.27717491737290945
 0.7228250826270906

ed = Indexed Edge 2 => 3 with index 4
ed_fourier = Indexed Edge 2 => 3 with index 4


In [314]:
node = 1
time = 3

edd = inedges(bp.g, node)
for ed in edd
    @show ed
    μ = bp.μ[idx(ed)]
    μ_marg = marginalize(μ,1)
    marg = marginals(μ_marg)[time]

    μ_fourier = bp_fourier.μ[idx(ed_fourier)]
    μ_fourier_marg = marginalize(μ_fourier,1)
    marg_fourier = marginals(μ_fourier_marg)[time]

    display(marg)
    display(marg_fourier)
end

2-element Vector{Float64}:
 0.00010539877535425973
 0.9998946012246457

2-element Vector{Float64}:
 0.27717491737290945
 0.7228250826270906

2-element Vector{Float64}:
 0.009861877518778988
 0.990138122481221

2-element Vector{Float64}:
 0.27717491737290945
 0.7228250826270906

2-element Vector{Float64}:
 0.06098540327776541
 0.9390145967222345

2-element Vector{Float64}:
 0.27717491737290945
 0.7228250826270906

ed = Indexed Edge 7 => 1 with index 21
ed = Indexed Edge 8 => 1 with index 28
ed = Indexed Edge 10 => 1 with index 34


In [315]:
for ed in edges(bp.g)
    display(ed)

    μ = bp.μ[idx(ed)]
    μ_marg = marginalize(μ,1)
    μ_fourier = bp_fourier.μ[idx(ed)]
    μ_fourier_marg = marginalize(μ_fourier,1)

    for time in 1:T+1
        marg = marginals(μ_marg)[time]
        marg_fourier = marginals(μ_fourier_marg)[time]
        display("time = $time")
        display(marg)
        display(marg_fourier)
    end
end

Indexed Edge 1 => 7 with index 1

"time = 1"

2-element Vector{Float64}:
 0.0
 1.0

2-element Vector{Float64}:
 3.225660680111042e-18
 1.0

"time = 2"

...

2-element Vector{Float64}:
 0.06574188900317428
 0.9342581109968257

2-element Vector{Float64}:
 0.09662719298994885
 0.9033728070100512

"time = 5"

2-element Vector{Float64}:
 0.06584282626019923
 0.9341571737398007

2-element Vector{Float64}:
 0.09346591567053807
 0.9065340843294619